In [1]:
import numpy as np

Vendor:  Continuum Analytics, Inc.
Package: mkl
Message: trial mode expires in 16 days


In [2]:


def remove_bad_chs(channel_groups, bad_ch_list):
    for k, v in channel_groups.items():
        for ch in v['channels']:
            if ch in bad_ch_list:
                idx = v['channels'].index(ch)
                v['channels'].pop(idx)
                v['geometry'].pop(ch)
    return channel_groups

def convert_sites_to_channels(site_groups, acq_channels_numbers, acq_site_mapping):
    """
    This maps sites to channels for geometries and the site lists. Also checks for integrity of site lists.
    
    site_groups is a dictionary with sites and geometries keys.
    acq_channel_numbers is a list of channels in order from the acquistion system (ie a range from 0-63).
    acq_site_mapping is a list of site numbers corresponding to the acquisition channel numbers specified above.
    
    
    So this will map:
        site_groups {
            0: {
            'sites': [46,45,44]
            }
        }
        acq_channel_numbers = [1,2,3]
        acq_site_mapping = [45, 46, 46]
        
        to:
        site_groups {
            0: {
            'sites': [3, 2, 1]
            }
        }
    """
    all_ch = []
    for k, v in site_groups.items():
        all_ch.extend(v['sites'])
    print ("{0} sites found on all groups.".format(len(all_ch)))
    bad = False
    for i in all_ch:
        nmatches = 0
        for ii in all_ch:
            if ii == i:
                nmatches += 1
        if nmatches >= 2:
            print ("ERROR: duplicate site of number {0} found.".format(i))
            bad = True
    if bad:
        return None
    else:
        print ("No duplicate channels.")
        
    for k, v in site_groups.items():
        site_list = v['sites']
        geo = v['geometry'].keys()
        
        assert len(site_list) == len(geo)
        
        for s in site_list:
            if s not in geo:
                raise ValueError('Site {0} specified in "sites" but not in "geometry" for shank {1}'.format(s, k))
        for s in geo:
            if s not in site_list:
                raise ValueError('Site {0} specified in "geometry" but not "sites" for shank {1}'.format(s, k))        
    print('Site and geometry lists are congruent.')
    
    channel_groups = {}
    for g, v in site_groups.items():
        channel_group = {}
        channels = []
        geometry = {}
        for s in v['sites']:
            site_idx = acq_site_mapping.index(s)
            ch = acq_channels_numbers[site_idx]
            channels.append(ch)
        channel_group['channels'] = channels
        
        for s, v in v['geometry'].items():
            site_idx = acq_site_mapping.index(s)
            ch = acq_channels_numbers[site_idx]
            geometry[ch] = v
        channel_group['geometry'] = geometry
        
        channel_groups[g] = channel_group
    return channel_groups

def linear_shank_graph(channel_list):
    """
    makes a graph to connect sites on a linear probe (ie Buszaki 64)
    """
    graph = []
    for i in range(len(channel_list)):
        a1 = channel_list[i]
        for ii in range(1, 3):
            try:
                a2 = channel_list[i+ii]
                graph.append((a1, a2))
            except:
                pass
    return graph

def make_graphs(channel_groups):
    for k in  channel_groups.keys():
        group = channel_groups[k]
        channels = group['channels']
        group['graph'] = linear_shank_graph(channels)
    return channel_groups

In [3]:
SGL_64 = {
    'raw_channels': range(64),
    'HIRES_4x16_flipchip': [
        18, 20, 21, 23, 24, 25, 26, 27, 31, 30, 29, 32, 28, 22, 19, 17, 48, 46, 43, 37, 33, 36, 35, 34, 38, 39, 40, 41,
        42, 44, 45, 47, 1, 3, 6, 8, 10, 12, 14, 16, 15, 13, 11, 9, 7, 5, 4, 2, 63, 61, 60, 58, 56, 54, 52, 50, 49, 51,
        53, 55, 57, 59, 62, 64
    ],
    'NanoZ': [
        34, 36, 38, 40, 42, 44, 46, 48, 47, 45, 43, 41, 39, 37, 35, 33, 57, 59, 62, 64, 50, 52, 54, 56, 55, 53,
        51, 49, 63, 61, 60, 58, 1, 3, 5, 7, 9, 11, 13, 15, 16, 14, 12, 10, 8, 6, 4, 2, 26, 28, 29, 31, 17, 19, 21,
        23, 24, 22, 20, 18, 32, 30, 27, 25
    ],
}

In [4]:
site_groups = {
    0: {
        'geometry':{
            56: (0, 0), 57: (5, 10), 55: (-6, 20), 58: (7, 30), 54: (-8, 40), 59: (9, 50), 
            53: (-10, 60), 60: (11, 70), 52: (-12, 80),61: (13, 90), 51: (-14, 100), 62: (15, 110), 
            50: (-16, 120), 63: (17, 130), 49: (-18, 140), 64: (19, 150)
        },
        'sites': [56, 57, 55, 58, 54, 59, 53, 60, 52, 61, 51, 62, 50, 63, 49, 64]
    },
    1: {
        'geometry': {
            41: (0, 0), 37: (5, 10), 33: (-6, 20), 42: (7, 30), 40: (-8, 40), 43: (9, 50), 36: (-10, 60), 44: (11, 70),
            39: (-12, 80), 45: (13, 90), 35: (-14, 100), 46: (15, 110), 38: (-16, 120), 47: (17, 130),
            34: (-18, 140), 48: (19, 150)
        },
        'sites': [41, 37, 33, 42, 40, 43, 36, 44, 39, 45, 35, 46, 38, 47, 34, 48]
    },
    2: {
        'geometry': {
            28: (0, 0), 24: (5, 10), 23: (-6, 20), 32: (7, 30), 22: (-8, 40),25: (9, 50), 21: (-10, 60),
            29: (11, 70), 20: (-12, 80), 26: (13, 90), 19: (-14, 100), 30: (15, 110), 18: (-16, 120),
            27: (17, 130), 17: (-18, 140), 31: (19, 150)
        },
        'sites': [28, 24, 23, 32, 22, 25, 21, 29, 20, 26, 19, 30, 18, 27, 17, 31]
    },
    3: {
        'geometry': {
            8: (0, 0), 9: (5, 10), 7: (-6, 20), 10: (7, 30), 6: (-8, 40), 11: (9, 50), 5: (-10, 60), 
            12: (11, 70), 4: (-12, 80), 13: (13, 90), 3: (-14, 100), 14: (15, 110), 2: (-16, 120),
            15: (17, 130), 1: (-18, 140), 16: (19, 150)
        },
        'sites': [8, 9, 7, 10, 6, 11, 5, 12, 4, 13, 3, 14, 2, 15, 1, 16]
    }
}

In [5]:
channel_groups = convert_sites_to_channels(site_groups, SGL_64['raw_channels'], SGL_64['HIRES_4x16_flipchip'])

64 sites found on all groups.
No duplicate channels.
Site and geometry lists are congruent.


In [6]:
NZ_bad = [1, 2, 4, 5, 20,]
def nz_to_sgl(nz_chs, acq_ch_map, nz_ch_map):
    ac_chs = []
    for ch in nz_chs:
        idx = nz_ch_map.index(ch)
        ac_chs.append(acq_ch_map[idx])
        ac_chs.sort()
    return ac_chs

a= nz_to_sgl(NZ_bad, SGL_64['raw_channels'], SGL_64['NanoZ'])
print (a)
print ([x+64 for x in a])

[32, 34, 46, 47, 58]
[96, 98, 110, 111, 122]


In [7]:
channel_groups = remove_bad_chs(channel_groups, [])

In [8]:
channel_groups = make_graphs(channel_groups)

In [13]:
channel_groups

{0: {'channels': [52,
   60,
   59,
   51,
   53,
   61,
   58,
   50,
   54,
   49,
   57,
   62,
   55,
   48,
   56,
   63],
  'geometry': {48: (17, 130),
   49: (13, 90),
   50: (11, 70),
   51: (7, 30),
   52: (0, 0),
   53: (-8, 40),
   54: (-12, 80),
   55: (-16, 120),
   56: (-18, 140),
   57: (-14, 100),
   58: (-10, 60),
   59: (-6, 20),
   60: (5, 10),
   61: (9, 50),
   62: (15, 110),
   63: (19, 150)},
  'graph': [(52, 60),
   (52, 59),
   (60, 59),
   (60, 51),
   (59, 51),
   (59, 53),
   (51, 53),
   (51, 61),
   (53, 61),
   (53, 58),
   (61, 58),
   (61, 50),
   (58, 50),
   (58, 54),
   (50, 54),
   (50, 49),
   (54, 49),
   (54, 57),
   (49, 57),
   (49, 62),
   (57, 62),
   (57, 55),
   (62, 55),
   (62, 48),
   (55, 48),
   (55, 56),
   (48, 56),
   (48, 63),
   (56, 63)]},
 1: {'channels': [27,
   19,
   20,
   28,
   26,
   18,
   21,
   29,
   25,
   30,
   22,
   17,
   24,
   31,
   23,
   16],
  'geometry': {16: (19, 150),
   17: (15, 110),
   18: (9, 50),
 

In [9]:
f = open('HIRES_1.prb', 'w')
# pickle.dump(channel_groups, f, 0)
f.write("channel_groups = {0}".format(channel_groups.__str__()))
f.close()

DOUBLE DOWN
---
Make a probe file for double 64 ch probes

In [17]:
dual_ch_groups = {}
group_shifts = (0,4)
ch_shifts = (0, 64)
for gshift, cshift in zip(group_shifts, ch_shifts):
    for k, v in channel_groups.items():
        channels = [x + cshift for x in v['channels']]
        geometry = {}
        for kk, vv in v['geometry'].items():
            geometry[kk + cshift] = vv
        graph = []
        for i in v['graph']:
            graph.append([x+cshift for x in i])
        dual_ch_groups[k+gshift] = {
            'channels': channels,
            'geometry': geometry,
            'graph': graph
        }
        

In [18]:
f = open('HIRES_dual.prb', 'w')
# pickle.dump(channel_groups, f, 0)
f.write("channel_groups = {0}".format(dual_ch_groups.__str__()))
f.close()

In [19]:
dual_ch_groups

{0: {'channels': [52,
   60,
   59,
   51,
   53,
   61,
   58,
   50,
   54,
   49,
   57,
   62,
   55,
   48,
   56,
   63],
  'geometry': {48: (17, 130),
   49: (13, 90),
   50: (11, 70),
   51: (7, 30),
   52: (0, 0),
   53: (-8, 40),
   54: (-12, 80),
   55: (-16, 120),
   56: (-18, 140),
   57: (-14, 100),
   58: (-10, 60),
   59: (-6, 20),
   60: (5, 10),
   61: (9, 50),
   62: (15, 110),
   63: (19, 150)},
  'graph': [[52, 60],
   [52, 59],
   [60, 59],
   [60, 51],
   [59, 51],
   [59, 53],
   [51, 53],
   [51, 61],
   [53, 61],
   [53, 58],
   [61, 58],
   [61, 50],
   [58, 50],
   [58, 54],
   [50, 54],
   [50, 49],
   [54, 49],
   [54, 57],
   [49, 57],
   [49, 62],
   [57, 62],
   [57, 55],
   [62, 55],
   [62, 48],
   [55, 48],
   [55, 56],
   [48, 56],
   [48, 63],
   [56, 63]]},
 1: {'channels': [27,
   19,
   20,
   28,
   26,
   18,
   21,
   29,
   25,
   30,
   22,
   17,
   24,
   31,
   23,
   16],
  'geometry': {16: (19, 150),
   17: (15, 110),
   18: (9, 50),
 

In [42]:
for i in a.keys():
    i += 1
a

{9: 45}